<a href="https://colab.research.google.com/github/DanielaManate/SentimentAnalysis_MADC_UBB_1NB/blob/master/SA_Final/SA_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
!pip install contractions

     |████████████████████████████████| 245kB 2.1MB/s 
     |████████████████████████████████| 317kB 45.9MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81697 sha256=282a02009d8618b3b52aaebd36b32e1e0f7202db03d7f9275c6c344146e75427
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [2]:
import pandas as pd
import numpy as np

# Lexicons
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Classification Report
from sklearn.metrics import classification_report

# Preprocessing
# Contractions
import contractions
# Regex 
import re
# NLTK
import nltk
# Stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
# Word tokenize
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# Stemmer
from nltk.stem import PorterStemmer

#BOW - Document-Term-Matrix
from sklearn.feature_extraction.text import CountVectorizer

# Train-Test-Split
from sklearn.model_selection import train_test_split
# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Picke
import pickle

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
path = 'gdrive/My Drive/SA_Final/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Reading Data

In [4]:
# Reading reviews
reviews = pd.read_csv(path + 'input_data.csv')
print(reviews.shape)
reviews.head(2)

(10346, 3)


,rest_id,text,rating
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0
1,yGMCl0vYigshkXiZFIDTNw,BOGO....I find that when a server is inefficie...,3.0


In [5]:
# Exclude rating of 3
reviews = reviews[reviews['rating']!=3].reset_index(drop=True)
print(reviews.shape)
reviews.head(2)

(9365, 3)


,rest_id,text,rating
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5.0


In [6]:
# Add column 'positive'
reviews['positive'] = np.where(reviews['rating']>3,1,0)
reviews.head(2)

,rest_id,text,rating,positive
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0,1
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5.0,1


# 1. Sentiment Analysis using Lexicon

In [7]:
reviews['pred_lexicon_score'] = reviews['text'].apply(lambda x: 
                                                    SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
reviews.head(2)

,rest_id,text,rating,positive,pred_lexicon_score
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0,1,0.8954
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5.0,1,0.7644


In [8]:
reviews['pred_lexicon'] = np.where(reviews['pred_lexicon_score']>=0, 1, 0)
reviews.head(2)

,rest_id,text,rating,positive,pred_lexicon_score,pred_lexicon
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0,1,0.8954,1
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5.0,1,0.7644,1


In [9]:
print(classification_report(reviews['positive'], 
                            reviews['pred_lexicon']))

              precision    recall  f1-score   support

           0       0.94      0.52      0.67      2839
           1       0.83      0.99      0.90      6526

    accuracy                           0.85      9365
   macro avg       0.88      0.76      0.79      9365
weighted avg       0.86      0.85      0.83      9365



In [10]:
cr_lexicon = pd.DataFrame(classification_report(reviews['positive'], 
                            reviews['pred_lexicon'], 
                            output_dict=True)).transpose()
cr_lexicon

,precision,recall,f1-score,support
0,0.941251,0.524833,0.673903,2839.000000
1,0.826651,0.985749,0.899217,6526.000000
accuracy,0.846022,0.846022,0.846022,0.846022
macro avg,0.883951,0.755291,0.786560,9365.000000
weighted avg,0.861392,0.846022,0.830913,9365.000000


In [0]:
cr_lexicon.to_csv(path + 'output_1lexicon.csv')

# 2. Sentiment Analysis using Classification

## Text Pre-Processing

In [0]:
def preprocess_text(document):
     # Replacing contractions (don't -> do not)
     document = contractions.fix(document)

     # Lowercasing
     document = document.lower()

     # Regex - https://www.regextester.com
     # ^ = NOT # \w = word # \s = whitespace
     document = re.sub(r'[^\w\s]', ' ', document)
     document = re.sub(r'[^a-z ]', '', document)

     # Tokenization (splitting document into words)
     document = word_tokenize(document)

     # Removing stopwords (common words; e.g. this, that, to, a)
     stop_words = stopwords.words('english')
     document = [word for word in document if word not in stop_words]

     # Stemmer
     document = [PorterStemmer().stem(word) for word in document]
     
     # Changing output from list to sentence
     document = ' '.join(document)

     return document

In [13]:
preprocess_text('test test2345 !!! ^ $ pre.process! absolute absolutely')

'test test pre process absolut absolut'

In [14]:
reviews['text_prep'] = [preprocess_text(document)
                for document in reviews['text'].to_list()]
reviews.head(2)

,rest_id,text,rating,positive,pred_lexicon_score,pred_lexicon,text_prep
0,yGMCl0vYigshkXiZFIDTNw,We arrived for our reservation at 7:15pm. The...,4.0,1,0.8954,1,arriv reserv pm seat us right time restur fair...
1,yGMCl0vYigshkXiZFIDTNw,We received amazing service again. The food wa...,5.0,1,0.7644,1,receiv amaz servic food cook right waitress sh...


## BOW

In [15]:
# Create dtm for most frequent 6000 words
vec = CountVectorizer(max_features=6000)
counts_per_review = vec.fit_transform(reviews['text_prep'])
review_words = vec.get_feature_names()
dtm = pd.DataFrame(counts_per_review.toarray(),
                   columns = review_words)
dtm.head(2)

,aaabar,aaron,abacu,abid,abil,abl,absolut,absurd,abund,ac,acai,accept,access,accid,accident,accommod,accomod,accompani,accomplish,accord,accordingli,accordion,account,accur,accuraci,accus,accustom,ace,ach,achiev,acid,acknowledg,acoust,acquir,across,act,action,activ,actual,ad,...,yell,yellow,yellowtail,yelp,yelper,yen,yep,yesterday,yet,yike,yo,yogurt,yoke,yolk,york,yorker,young,younger,youngster,youth,yr,yuck,yum,yumm,yummi,yummm,yummo,yup,zach,zack,zero,zesti,zillion,zipp,ziti,zoe,zone,zucchini,zuchinni,zupa
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Machine Learning Approach

In [0]:
# Split into training & test set
X_train, X_test, y_train, y_test = train_test_split(
    dtm,
    reviews['positive'],
    train_size = 0.8,
    random_state = 5
)

In [17]:
%%time
log_model = LogisticRegression(solver = 'sag', max_iter = 3000,
                                   random_state = 5)
log_model.fit(X_train, y_train)

CPU times: user 3min 22s, sys: 168 ms, total: 3min 22s
Wall time: 3min 22s


In [0]:
# Get predictions for test set
y_pred = log_model.predict(X_test)

In [19]:
# Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       528
           1       0.96      0.96      0.96      1345

    accuracy                           0.94      1873
   macro avg       0.93      0.93      0.93      1873
weighted avg       0.94      0.94      0.94      1873



In [20]:
cr_ml = pd.DataFrame(classification_report(y_test, 
                            y_pred, 
                            output_dict=True)).transpose()
cr_ml

,precision,recall,f1-score,support
0,0.900952,0.895833,0.898386,528.000000
1,0.959199,0.961338,0.960267,1345.000000
accuracy,0.942872,0.942872,0.942872,0.942872
macro avg,0.930076,0.928586,0.929326,1873.000000
weighted avg,0.942779,0.942872,0.942823,1873.000000


In [0]:
cr_ml.to_csv(path + 'output_2ml.csv')